# Assigment 6: APIs

I used the SF OpenData portal API to upload the SF Garage and Lot offstreet parking. The following code details the extraction and importation of the dataset.

In [1]:
import pandas as pd
import urllib
import json

In [2]:
# used endpoint for Off-street parking lots and parking garages
endpoint_url = 'https://data.sfgov.org/resource/uupn-yfaw.json'

# open connection to the URL
connection = urllib.urlopen(endpoint_url)

# downloading the results
results = connection.read()

# parsing the string into a Python data structure
data = json.loads(results)

In [3]:
# Checking to see how the data looks like..
data[0]

{u'address': u'2110 Market St',
 u'garorlot': u'L',
 u'landusetyp': u'restaurant',
 u'location_1': {u'latitude': u'37.767378',
  u'longitude': u'-122.429344',
  u'needs_recoding': False},
 u'mccap': u'0',
 u'owner': u'Private',
 u'primetype': u'PPA',
 u'regcap': u'13',
 u'secondtype': u' ',
 u'valetcap': u'0'}

In [4]:
# code turns the json data into a dataframe
df = pd.DataFrame(data)
df.head()

,address,garorlot,landusetyp,location_1,mccap,owner,primetype,regcap,secondtype,valetcap
0,2110 Market St,L,restaurant,"{u'latitude': u'37.767378', u'needs_recoding':...",0,Private,PPA,13,,0
1,993 Potrero,L,,"{u'latitude': u'37.757272', u'needs_recoding':...",0,SFMTA,PPA,34,,0
2,601 Terry A Francois Blvd,L,,"{u'latitude': u'37.770135', u'needs_recoding':...",0,Port of SF,PPA,72,,0
3,11 SOUTH VAN NESS,G,,"{u'latitude': u'37.77415', u'needs_recoding': ...",0,Private,PHO,130,CPO,0
4,101 CALIFORNIA ST,G,,"{u'latitude': u'37.793243', u'needs_recoding':...",0,Private,PPA,250,,0


In [5]:
# Extracting the coordinates from the location attributes.
df['latitude'] = df['location_1'].apply(lambda x: x['latitude'])
df['longitude'] = df['location_1'].apply(lambda x: x['longitude'])

In [6]:
print len(df) # size of the dataset
df.head()

1000


,address,garorlot,landusetyp,location_1,mccap,owner,primetype,regcap,secondtype,valetcap,latitude,longitude
0,2110 Market St,L,restaurant,"{u'latitude': u'37.767378', u'needs_recoding':...",0,Private,PPA,13,,0,37.767378,-122.429344
1,993 Potrero,L,,"{u'latitude': u'37.757272', u'needs_recoding':...",0,SFMTA,PPA,34,,0,37.757272,-122.40632
2,601 Terry A Francois Blvd,L,,"{u'latitude': u'37.770135', u'needs_recoding':...",0,Port of SF,PPA,72,,0,37.770135,-122.385801
3,11 SOUTH VAN NESS,G,,"{u'latitude': u'37.77415', u'needs_recoding': ...",0,Private,PHO,130,CPO,0,37.77415,-122.418402
4,101 CALIFORNIA ST,G,,"{u'latitude': u'37.793243', u'needs_recoding':...",0,Private,PPA,250,,0,37.793243,-122.397495


In [7]:
#Checking the unique landuse types near the parking garages and/or lots
df.landusetyp.unique()

array([u'restaurant', u' ', u'grocery', u'religious', u'service',
       u'medical', u'automotive', u'office', u'entertainment', u'hotel',
       u'retail', u'government'], dtype=object)

In [8]:
# Checking for the percentage of rows with landuse type
print len(df[df.landusetyp != ' '])* 100.0/len(df)

16.3


In [9]:
# Given that only 16% of the rows have this variable, I decided to remove that variable
ds = df[['address', 'garorlot', 'owner', 'primetype', 'regcap', 'valetcap', 'latitude', 'longitude']]

In [10]:
ds.describe()

,address,garorlot,owner,primetype,regcap,valetcap,latitude,longitude
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,988,4,24,5,258,25,990,990
top,,L,Private,PPA,40,0,37.79811,-122.397471
freq,5,649,829,484,35,957,2,2


In [11]:
ds.owner.unique()

array([u'Private', u'SFMTA', u'Port of SF', u'RPD', u'UCSF', u'SFRA',
       u'GGNRA', u'GG Bridge Authority', u'Presidio', u'UC', u'SFSU',
       u'BART', u'Caltrans', u'Customs', u'SFPUC', u'SFPD', u'USPS',
       u'City College', u'LHH', u'SFGH', u'SFUSD', u'State of CA',
       u'SF Sheriff', u'DMV'], dtype=object)

In [12]:
ds.primetype.unique()

array([u'PPA', u'PHO', u'CPO', u' ', u'CGO'], dtype=object)

In [13]:
# Checking for the percentage of rows without primetype code
print len(df[df.primetype == ' '])* 100.0/len(df)

0.2


In [14]:
ds.garorlot.unique()

array([u'L', u'G', u' ', u'GL'], dtype=object)

In [15]:
# Checking for the percentage of rows without specified garage or lot
print len(df[df.garorlot == ' '])* 100.0/len(df)

3.1


In [16]:
ds.loc[:,'regcap'] = ds['regcap'].astype('float64').astype('int')
ds.loc[:,'valetcap'] = ds['valetcap'].astype('float64').astype('int')
ds.loc[:,'latitude'] = ds['latitude'].astype('float64')
ds.loc[:,'longitude'] = ds['longitude'].astype('float64')

/Users/Alan/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pandas/core/indexing.py:490: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/Users/Alan/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [17]:
ds['address'] = df['address'].apply(lambda text: text.encode('utf-8'))

/Users/Alan/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [18]:
ds[['regcap', 'valetcap', 'latitude', 'longitude']].describe()

,regcap,valetcap,latitude,longitude
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,137.294000,3.297000,37.773081,-122.418183
std,361.051209,22.624824,0.022588,0.025672
min,0.000000,0.000000,37.708475,-122.505057
25%,24.000000,0.000000,37.761489,-122.430397
50%,56.000000,0.000000,37.779071,-122.411403
75%,132.000000,0.000000,37.788860,-122.399540
max,9000.000000,430.000000,37.809485,-122.371834


In [19]:
ds.head()

,address,garorlot,owner,primetype,regcap,valetcap,latitude,longitude
0,2110 Market St,L,Private,PPA,13,0,37.767378,-122.429344
1,993 Potrero,L,SFMTA,PPA,34,0,37.757272,-122.406320
2,601 Terry A Francois Blvd,L,Port of SF,PPA,72,0,37.770135,-122.385801
3,11 SOUTH VAN NESS,G,Private,PHO,130,0,37.774150,-122.418402
4,101 CALIFORNIA ST,G,Private,PPA,250,0,37.793243,-122.397495


In [20]:
ds.to_csv('OffstParkingSF.csv')